# We are going to enter 'Paddy Doctor: Paddy Disease Classification'

<a href="https://www.kaggle.com/competitions/paddy-disease-classification">Paddy disease</a> Here is the competition

Go to the competition data page and <b>agree to the terms of service</b>. Once you do that you can get data and submit results. At the bottom of the page there is a kaggle CLI link to download the data, after installing the CLI (see below) get this link and run it.

# Kaggle CLI: Install and set up auth token

This makes it really easy to get data and submit results.  Plus it makes it easy to automate tedious stuff (like downloading data)<br>


In [ ]:
#first ensure kaggle is installed
#will have to run this everytime unless pip install --user kaggle, but then may run into problems when using ctags to index the fastai user library
try:
    import kaggle
except ModuleNotFoundError:
    #not there install
    !pip install kaggle

#if kaggle is installed but you dont have key installed then importing kaggle will throw an exception
#so upload key (verify it's permissions BTW)
#to get key- go to Settings-Account-API-Create new Token
!chmod 600 /root/.kaggle/kaggle.json

## Constants and some useful functions

In [ ]:
TRAIN_DIR = './train_images'
TEST_DIR='./test_images'
NUMB_OUTPUT_CLASSES_TO_PREDICT=10
DEFAULT_BATCH_SIZE=64
DEFAULT_LR=0.001

#!pip install pynvml
import gc
import torch
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()
report_gpu()

#run this in another terminal window to see what is happenning 
#to your gpu memory(assumming you have one).  It will update every 2 seconds
#watch nvidia-smi

defaults, if not using all GPU memory increate batch size and learningrate<br>
Rule of thumb: if double bs multiply lr by 1.4

In [ ]:
bs=DEFAULT_BATCH_SIZE*2
lr=DEFAULT_LR*1.1

## Kaggle CLI: Get the Data

In [ ]:
#download data once
import os
if not os.path.exists(TRAIN_DIR):
    #get the data
    !kaggle competitions download -c paddy-disease-classification
    
    #unzip it, -q is silent
    !unzip -q paddy-disease-classification.zip
    
    #remove orig zip file (because its large)
    !rm paddy-disease-classification.zip

    #it put it all in /notebooks, thats fine if you dont mind importing it every time you start a machine 
    #(it's pretty fast though so why not?) Otherwise move it to storage and symlink it back, if you do this 
    #keep in mind you are permanetly taking up 1GB of your allocated 20GB of space

## Explore data

Notice that train_images has 10 folders, each with a picture of rice with a type of disease
<br><b>But test_images has no folders. That's because you have to determine which label to apply to each image

In [ ]:
#what is train.csv?
import pandas as pd
df =pd.read_csv('train.csv')
df.head()

In [ ]:
#it has a list of all the training files and their labels
df.label.value_counts()

The dataset is imbalanced, easy to address (use a weighted dataloader in training) but beyond the scope of this course

Variety and age both have usefull info, but using them will complicate our model quite a bit, so we will ignore them for now.

### Turns out some of the images are different sizes, each batch has to have images the same size

see what the sizes are and remove the (640,480) images

In [ ]:
from tqdm import tqdm
from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.all import *

In [ ]:
from fastcore.parallel import *

fls=get_image_files(TRAIN_DIR)
def f(o): return PILImage.create(o).size
sizes = parallel(f, fls, n_workers=6)
pd.Series(sizes).value_counts()

### Remove them?  We could but what if the test set also has different sizes (it does)

Lets just resize them when needed

## OK Lets build a model

### First load the Data

Accomplished using dataloaders, a dataloader boils down to 2 parts: a dataset and a Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples

#### BTW Experimenting goes much (MUCH!) faster if you use a subset of original data

How about 50 of each?  Dataset size is roughly 100 then.  Not the same distribution as original dataset but faster to verify that things are working before we actually commit to the full dataset

In [ ]:
#lets get 50 of each type
new_df = df.groupby('label').sample(n=50)
new_df

#### Method 1- Load all data (using file system)

dataloaders are flexible, here I'm telling it where the images are and each folder name will be interpreted as the contained images label.



In [ ]:
# if you dont have a valid_pct then you get a Nonetype object is non iterable error
# dls=ImageDataLoaders.from_folder('/notebooks/train_images',valid_pct=0.2)
dls=ImageDataLoaders.from_folder('/notebooks/PCSE_595_code/train_images',valid_pct=0.2,bs=bs,item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=128, min_scale=0.75))
# # dls.show_batch()
# dls.train_ds.items[:3]

#if you want to get rid of
# del(dls)
# report_gpu()

In [ ]:
dls.show_batch()
# dls.train_ds.items[:3]

#### Method 2- Load all data using the .csv file

Maybe use a smaller dataset to verify training?

In [ ]:
#create a new column which indicates where each image is located in TRAIN_DIR
df["loc"] = df['label'].astype(str) +"/"+ df["image_id"]
df

In [ ]:
#when satisfied that things are set up properly, train on full dataset(df) instead of new_df
dls = ImageDataLoaders.from_df(df=df, #df=new_df or df
                               path=TRAIN_DIR, 
                               fn_col=4,
                               label_col=1,
                               suff='', #do not add extra .jpg
                               folder=".",
                               bs=bs,
                              valid_pct=0.2,item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=128, min_scale=0.75))
# dls.show_batch()

In [ ]:
#clean up any memory that we can
report_gpu()

### Next Choose a timm Model to Fine Tune

Which one though?  Look at <a href="https://www.kaggle.com/code/jhoward/the-best-vision-models-for-fine-tuning">The best vision models for fine-tuning</a>.  This notebook compares many different models which are fine-tuned on two differnt datasets;  <a href="https://www.robots.ox.ac.uk/~vgg/data/pets/"> Oxford IIT-Pet</a> and then <a href="https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/data">Kaggle planet (space images)</a>.  

<mark>Choose a model with a low error rate and fit time (trains faster) on the type of data you are using.</mark>  For instance, if you want to build a dogs verses cats classifier, then choose a model that performed well when fine tuned on <a href="https://www.robots.ox.ac.uk/~vgg/data/pets/"> Oxford IIT-Pet</a>.

<b>BTW You want a pretrained model to fine tune on, so just look at the pretrained ones

In [ ]:
#what 
import timm
# timm.list_models(pretrained=True)  #all models
timm.list_models(['*convnext_tiny*'],pretrained=True)  #pretrained convnext models

### Now train the model

The model is wrapped in a learner object which also takes our dataloader

In [ ]:
# Use vim -t vision_learner to see where fastai creates either a timm or a pytorch model

In [ ]:
#this learner will use pytorches convnext_tiny (type convnext_tiny?? in cell to get location)
# learn = vision_learner(dls, convnext_tiny, metrics=error_rate, path='.')

#this learner will use timms convnext_tiny model
learn = vision_learner(dls, 'convnext_tiny',lr=lr, metrics=error_rate, path='.')

In [ ]:
learn.fine_tune(5)
# learn.fit_one_cycle(3)

In [ ]:
# check out what is happenning to GPU memory (watch nvidia-smi) is it being used?
#if we have a lot of extra memory then increase the batch size to increase training speed
#(rule of thumb: also increase learning rate by roughly square root of the batch size multiplier
# so if you quadruple batch size, multiply learning rate by 2)

### Can we optimize anything? Speed stuff up?

see how much gpu is being used nvidia-smi

how much memory is being used (install htop)

turns out we are not really using much of our GPU, lets double the batch size and 1.4*lr


In [ ]:
print(f'current batch size={learn.data.bs}, current learning rate={learn.lr}')

In [ ]:
#increase batch size of images to go to the gpu at a time
learn.data.bs=learn.data.bs*2

#increase model learning rate
learn.lr=learn.lr*1.4

### Save model for later

In [ ]:
#just save the model to weights dir (its pretty big)
#want to see how it works? Go to def of save (vim -t save) and find where it actualy saves the model (save_model function) 
learn.save('weights_only', with_opt=True)

In [ ]:
# learn.load('weights_only', with_opt=True)

### Use model to generate submission

Note: This part comes from  <a href="https://www.kaggle.com/code/jhoward/first-steps-road-to-the-top-part-1">First Steps: Road to the Top, Part 1</a>.  An excellent series showing increasingly better submissions

In [ ]:
#first what does ss look like
ss = pd.read_csv('./sample_submission.csv')
ss

In [ ]:
#load the test files
tst_files = get_image_files(TEST_DIR).sorted()
tst_dl = dls.test_dl(tst_files)

In [ ]:
#We can now get the probabilities of each class, and the index of the most likely class, from this test set (the 2nd thing returned by get_preds are the targets, which are blank for a test set, so we discard them):
probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)
idxs

These need to be mapped to the names of each of these diseases, these names are stored by fastai automatically in the vocab:

In [ ]:
dls.vocab

We can create an apply this mapping using pandas:

In [ ]:
mapping = dict(enumerate(dls.vocab))
results = pd.Series(idxs.numpy(), name="idxs").map(mapping)
results

Kaggle expects the submission as a CSV file, so let's save it, and check the first few lines:

In [ ]:
ss['label'] = results
ss.to_csv('subm.csv', index=False)
!head subm.csv

### Zip And Upload to Kaggle

In [ ]:
!zip ./subm.zip ./subm.csv
!kaggle competitions submit -c paddy-disease-classification -f subm.zip -m "Message"

## TestBed

In [ ]:
#NOTE if you run the model below you will get a runtime error that says at least 2 tensors are not the same size
#lets see what the sizes are

# from collections import defaultdict
# szs=defaultdict(int)
# for fle in tqdm(fls):
#     szs[Image.open(fle).size]+=1
# szs

#delete the few missized images
# import os
# for fle in tqdm(fls):
#     if( Image.open(fle).size == (640,480)):
#         os.remove(fle)

#NOTE if you leave the data in the notebooks folder then its permanant so 
# YOU ONLY HAVE TO RUN THIS EVERY TIME YOU DOWNLOAD THE DATA